In [ ]:
# 📌 Cell 1: Imports
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.optimizers import Adam

In [ ]:
# 📌 Cell 2: Dataset paths
violence_path = "Violence"
nonviolence_path = "NonViolence"

IMG_SIZE = 112     # resize frames
SEQUENCE_LEN = 20  # no. of frames per video

In [ ]:
# 📌 Cell 3: Function to load videos
def load_videos_from_folder(folder, label):
    X, y = [], []
    for file in os.listdir(folder):
        file_path = os.path.join(folder, file)
        cap = cv2.VideoCapture(file_path)
        frames = []
        while len(frames) < SEQUENCE_LEN and cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
            frame = frame / 255.0
            frames.append(frame)
        cap.release()
        if len(frames) == SEQUENCE_LEN:
            X.append(frames)
            y.append(label)
    return X, y

In [ ]:
# 📌 Cell 4: Collect video file paths instead of loading everything into memory

# Get all Violence and NonViolence video file paths
violence_files = [os.path.join(violence_path, f) for f in os.listdir(violence_path) if f.endswith(".mp4")]
nonviolence_files = [os.path.join(nonviolence_path, f) for f in os.listdir(nonviolence_path) if f.endswith(".mp4")]

# Labels: 1 = Violence, 0 = NonViolence
video_paths = violence_files + nonviolence_files
labels = [1] * len(violence_files) + [0] * len(nonviolence_files)

print("Total videos found:", len(video_paths))
print("Violence videos:", len(violence_files))
print("NonViolence videos:", len(nonviolence_files))


In [ ]:
# 📌 Cell 5: Train-test split (only paths & labels)

from sklearn.model_selection import train_test_split

train_paths, test_paths, y_train, y_test = train_test_split(
    video_paths, labels, test_size=0.2, random_state=42
)

print("Training videos:", len(train_paths))
print("Testing videos:", len(test_paths))

In [ ]:
# 📌 Cell 6: Data Generator + Model definition

from tensorflow.keras.utils import Sequence, to_categorical

# ====== Data Generator ======
class VideoDataGenerator(Sequence):
    def __init__(self, video_paths, labels, batch_size=8, sequence_len=20, img_size=112, n_classes=2):
        self.video_paths = video_paths
        self.labels = labels
        self.batch_size = batch_size
        self.sequence_len = sequence_len
        self.img_size = img_size
        self.n_classes = n_classes

    def __len__(self):
        return int(np.ceil(len(self.video_paths) / self.batch_size))

    def __getitem__(self, idx):
        batch_paths = self.video_paths[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_labels = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]

        X, y = [], []
        for path, label in zip(batch_paths, batch_labels):
            cap = cv2.VideoCapture(path)
            frames = []
            while len(frames) < self.sequence_len and cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break
                frame = cv2.resize(frame, (self.img_size, self.img_size)).astype("float32") / 255.0
                frames.append(frame)
            cap.release()

            if len(frames) == self.sequence_len:   # only keep valid sequences
                X.append(frames)
                y.append(label)

        return np.array(X, dtype="float32"), to_categorical(y, self.n_classes)

# ====== Model Definition ======
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TimeDistributed, GlobalAveragePooling2D, LSTM, Dropout, Dense
from tensorflow.keras.optimizers import Adam

base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(112, 112, 3))

model = Sequential([
    TimeDistributed(base_model, input_shape=(20, 112, 112, 3)),
    TimeDistributed(GlobalAveragePooling2D()),
    LSTM(64),
    Dropout(0.5),
    Dense(64, activation="relu"),
    Dense(2, activation="softmax")
])

model.compile(loss="categorical_crossentropy", optimizer=Adam(1e-4), metrics=["accuracy"])
model.summary()


In [ ]:
# 📌 Cell 7: Training with generators

# Create generators
train_gen = VideoDataGenerator(train_paths, y_train, batch_size=8, sequence_len=20, img_size=112)
test_gen  = VideoDataGenerator(test_paths, y_test, batch_size=8, sequence_len=20, img_size=112)

# Train model
history = model.fit(
    train_gen,
    validation_data=test_gen,
    epochs=10,
    verbose=1
)

# Save model
model.save("violence_model.h5")

print("✅ Training complete. Model saved as violence_model.h5")


In [ ]:
import cv2, numpy as np, os
from tensorflow.keras.models import load_model
import requests

# ===== Load model =====
model = load_model("violence_model.h5")

IMG_SIZE = 112
SEQUENCE_LEN = 20

def predict_and_popup(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while len(frames) < SEQUENCE_LEN and cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_resized = cv2.resize(frame, (IMG_SIZE, IMG_SIZE)).astype("float32") / 255.0
        frames.append(frame_resized)
    cap.release()

    label = "Video too short"
    if len(frames) == SEQUENCE_LEN:
        frames = np.expand_dims(frames, axis=0)
        pred = model.predict(frames)

        # Show raw predictions for debugging
        print("Raw prediction:", pred)

        # ⚠ Fix: swapped index mapping
        label = "Violence" if np.argmax(pred) == 0 else "NonViolence"

        # Trigger popup only for Violence
        if label == "Violence":
            requests.get(f"http://127.0.0.1:5000/test/{os.path.basename(video_path)}")

    # ===== Show overlay =====
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        color = (0,0,255) if label=="Violence" else (0,255,0)
        cv2.putText(frame, f"Prediction: {label}", (20, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.2, color, 3)
        cv2.imshow("Prediction Result", frame)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    return label

# ===== Test Example =====
print("Prediction:", predict_and_popup("TestVideos/vt1.mp4"))


In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import requests
from collections import deque

# ===== Load model =====
model = load_model("violence_model.h5")

IMG_SIZE = 112
SEQUENCE_LEN = 20

# Use deque to maintain a sliding window of frames
frames = deque(maxlen=SEQUENCE_LEN)

# ===== Start webcam =====
cap = cv2.VideoCapture(0)  # 0 = default laptop webcam

if not cap.isOpened():
    print("Cannot open webcam")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    # Preprocess frame
    frame_resized = cv2.resize(frame, (IMG_SIZE, IMG_SIZE)).astype("float32") / 255.0
    frames.append(frame_resized)

    label = "Waiting..."
    color = (255, 255, 0)

    # Predict only if we have enough frames
    if len(frames) == SEQUENCE_LEN:
        input_frames = np.expand_dims(np.array(frames), axis=0)
        pred = model.predict(input_frames)
        # ⚠ Fix: check your model's output order
        label = "Violence" if np.argmax(pred) == 0 else "NonViolence"

        # Trigger popup only for Violence
        if label == "Violence":
            try:
                requests.get("http://127.0.0.1:5000/test/live")  # adjust your endpoint
            except:
                pass
        color = (0, 0, 255) if label == "Violence" else (0, 255, 0)

    # Overlay prediction
    cv2.putText(frame, f"Prediction: {label}", (20, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, color, 3)
    cv2.imshow("Live Violence Detection", frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
